In [1]:
from math import pi

import pandas as pd

from bokeh.io import show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.palettes import viridis
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import picaso.picaso_data as ptest
import picaso.disort_disco as dtest

/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [3]:
# Toon quadrature
Toon_quad = ptest.picaso_albedos(output_dir='Toon_quad.csv', delta_eddington=True)
# Toon eddington
Toon_edd = ptest.picaso_albedos(output_dir= 'Toon_eddd.csv',Toon_coefficients ="eddington", delta_eddington=True)
# SH 2-stream
SH2 = ptest.picaso_albedos(output_dir='SH2.csv',method="SH", delta_eddington=True)
# SH 4-stream
SH4 = ptest.picaso_albedos(output_dir='SH4.csv',method='SH', stream=4, delta_eddington=True)

YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT


In [20]:
# disort = dtest.disort_albedos(output_dir='disort.csv')

In [4]:
# read in data from picaso function
data_old = pd.read_csv('DLUGACH_TEST.csv',index_col=0)
data_disort = pd.read_csv('disort.csv',index_col=0)
data_SH2 = pd.read_csv('SH2.csv',index_col=0)
data_SH4 = pd.read_csv('SH4.csv',index_col=0)
data_quad = pd.read_csv('Toon_quad.csv',index_col=0)
data_edd = pd.read_csv('Toon_eddd.csv',index_col=0)

#If you want to compare two different files for the heatmap just do this
data_compare = data_old
compare_quad = (data_compare-data_quad)/data_compare*100   
compare_edd = (data_compare-data_edd)/data_compare*100   
compare_SH2 = (data_compare-data_SH2)/data_compare*100 
compare_SH4 = (data_compare-data_SH4)/data_compare*100    

In [5]:
from bokeh.palettes import RdGy
def create_heat_map(data):
    data.columns.name = 'g0' 
    data.index.name = 'w0' 
    data.index=data.index.astype(str)
    data = data.rename(index={"-1.0":"Ray"})


    w0s = list(data.index)
    g0s = list(data.columns)

    # reshape to 1D array or rates with a month and year for each row.
    df = pd.DataFrame(data.stack(), columns=['albedo']).reset_index()

    colors = RdGy[11]
    bd = max(abs(df.albedo.min()), abs(df.albedo.max()))
    bd = min(bd,50)
    mapper = LinearColorMapper(palette=colors, low=-bd, high=bd)

    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

    p = figure(height=300,width=300,
           x_range=w0s, y_range=list(reversed(g0s)),
           x_axis_location="above",
           tools=TOOLS, toolbar_location='below')

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "7px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    p.rect(x="w0", y="g0", width=1, height=1,
       source=df,
       fill_color={'field': 'albedo', 'transform': mapper},
       line_color=None)

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12px",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     label_standoff=6, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    p.axis.major_label_text_font_size='12px'
    return p

In [6]:
show(create_heat_map(compare_quad))

In [7]:
show(create_heat_map(compare_SH2))

In [8]:
show(create_heat_map(compare_SH4))

In [9]:
data_SH4

,1.000,0.999,0.995,0.990,0.980,0.950,0.900,0.800,0.700
Unnamed: 0,,,,,,,,,
-1.00,0.736644,0.684940,0.625190,0.584426,0.531843,0.442005,0.358539,0.263127,0.202493
0.00,0.689000,0.636000,0.575000,0.534000,0.482000,0.392000,0.309000,0.218000,0.162000
0.50,0.640154,0.573845,0.500964,0.452513,0.391765,0.293442,0.209920,0.126786,0.082726
0.75,0.642000,0.537000,0.431000,0.366000,0.290000,0.182000,0.108000,0.052000,0.030000
0.80,0.640000,0.524000,0.410000,0.341000,0.262000,0.157000,0.088000,0.040000,0.023000
0.85,0.639000,0.499000,0.381000,0.308000,0.228000,0.126000,0.067000,0.029000,0.016000
0.90,0.638000,0.481000,0.338000,0.260000,0.180000,0.090000,0.043000,0.018000,0.010000


In [10]:
data_old

g0,1.000,0.999,0.995,0.990,0.980,0.950,0.900,0.800,0.700
w0,,,,,,,,,
-1.00,0.751,0.697,0.637,0.595,0.542,0.451,0.365,0.268,0.205
0.00,0.689,0.636,0.575,0.534,0.482,0.392,0.309,0.218,0.162
0.50,0.649,0.574,0.493,0.440,0.374,0.271,0.187,0.108,0.069
0.75,0.642,0.537,0.431,0.366,0.290,0.182,0.108,0.052,0.030
0.80,0.640,0.524,0.410,0.341,0.262,0.157,0.088,0.040,0.023
0.85,0.639,0.499,0.381,0.308,0.228,0.126,0.067,0.029,0.016
0.90,0.638,0.481,0.338,0.260,0.180,0.090,0.043,0.018,0.010


In [11]:
compare_quad

g0,1.000,0.999,0.995,0.990,0.980,0.950,0.900,0.800,0.700
w0,,,,,,,,,
-1.0,0.073548,1.607824e-01,5.314051e-01,0.579537,7.934240e-01,1.007050,7.801521e-01,8.086265e-01,0.243530
0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,1.151684e-14,0.000000,0.000000e+00,0.000000e+00,0.000000
0.5,0.933266,7.053758e-01,3.912205e-01,0.138340,-4.000658e-01,-1.267653,-2.325036e+00,-3.745735e+00,-4.190964
0.75,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,1.284980e-14,0.000000e+00,0.000000
0.8,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000
0.85,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,-1.196361e-14,0.000000
0.9,0.000000,1.154078e-14,-1.642342e-14,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000
